In [263]:
os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/15.RecruitRestarauntPrediction')
%run 00.Scripts/03.WeeklyPrediction.py

In [281]:
#get_group_model(['cluster']).to_csv('03.Profile/03.WeeklyPredictions1.csv')
get_group_model(['cluster','area_type']).to_csv('03.Profile/03.WeeklyPredictions2.csv')
get_group_model(['cluster','cuisine']).to_csv('03.Profile/03.WeeklyPredictions3.csv')
get_group_model(['cluster','area_type','cuisine']).to_csv('03.Profile/03.WeeklyPredictions4.csv')

ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [269]:
store['air_visit_data'].head()

visitors
air_store_id         visit_date          
air_ba937bf13d40fb24 2016-01-13        25
                     2016-01-14        32
                     2016-01-15        29
                     2016-01-16        22
                     2016-01-18         6

In [280]:
def get_ts_values(ds,period,label,reverse=False):
    weekly_tgts=[]
    for i,j in enumerate(ds.columns.get_level_values(-1)):
        if reverse==True:
            b=ds.iloc[:,i:min(i+period,ds.shape[1])]
        else: 
            b=ds.iloc[:,max(i-period,0):i]
        b.columns=['%s_%d'%(label,x) for x in range(b.shape[1])]
        b.loc[:,'week_start']=j
        weekly_tgts.append(b.set_index('week_start',append=True))
    return pd.concat(weekly_tgts)

def get_reserve_vars(pdate,group_level,c):
    c1=c[(c['reserve_week']<pdate)&(c['visit_week']>=pdate)]
    c2=c1.groupby(group_level+['visit_week'])['hpg'].sum().unstack()
    c2.columns=['hpg_reserve_%d'%((x-pdate).days/7) for x in c2.columns]
    c2['week_start']=pdate
    reserve1=c2.set_index('week_start',append=True)
    c2=c1.groupby(group_level+['visit_week'])['air'].sum().unstack()
    c2.columns=['air_reserve_%d'%((x-pdate).days/7) for x in c2.columns]
    c2['week_start']=pdate
    reserve2=c2.set_index('week_start',append=True)
    return reserve1,reserve2
    
def get_group_model(group_level):
    a=pd.merge(store['air_visit_data'].reset_index(),store['store_info'].reset_index(),on='air_store_id')
    a['week_start']=a['visit_date']-pd.to_timedelta((a['visit_date'].dt.weekday+1)%7,unit='D')
    weekly_visits=a[a['week_start']>=date(2016,6,1)]
    weekly_visits=weekly_visits.groupby(group_level+['week_start'])['visitors'].sum().to_frame('weekly_visits').unstack()

    tgts=get_ts_values(weekly_visits,6,'tgt_visitors',reverse=True)
    pvar=get_ts_values(weekly_visits,6,'prev_visits')
    cal_vars=get_ts_values(cal.T,6,'num_holidays',reverse=True).fillna(0)
    cal_vars.index=cal_vars.index.get_level_values(1)

    c=store['reserve_master'].copy()
    c['reserve_date']=c['reserve_date'].astype(np.datetime64)
    c['visit_date']=c['visit_date'].astype(np.datetime64)
    c['reserve_week']=c['reserve_date']-pd.to_timedelta((c['reserve_date'].dt.weekday+1)%7,unit='D')
    c['visit_week']=c['visit_date']-pd.to_timedelta((c['visit_date'].dt.weekday+1)%7,unit='D')
    c['interval']=(c['visit_week']-c['reserve_week']).dt.days/7
    c=c[(c['interval']>0)&(c['interval']<=6)]
    c=c.groupby(group_level+['reserve_week','visit_week','interval','type'])['reserve_visitors'].sum().unstack().reset_index()

    data=[get_reserve_vars(pdate,group_level,c) for pdate in pd.concat([tgts,pvar]).index.get_level_values(-1).drop_duplicates()]
    reserve1,reserve2=zip(*data)
    reserve1=pd.concat(reserve1)
    reserve2=pd.concat(reserve2)

    master=pd.concat([tgts,pvar,reserve1,reserve2,cal_vars.reindex(tgts.index, level=1)],axis=1)
    test_vars1=weekly_visits.iloc[:,-6:]
    test_vars1.columns=['prev_visits_%d'%(5-i) for i in range(6)]
    test_reserve_1,test_reserve_2=get_reserve_vars(datetime(2017,4,23),group_level,c)
    test_reserve_1.reindex(test_vars1)
    test_reserve_2.reindex(test_vars1)
    test_data=pd.concat([test_vars1,test_reserve_1,test_reserve_2],axis=1).fillna(0)

    for i in range(6):
        test_data['num_holidays_%d'%i]=cal.loc[date(2017,4,23)+timedelta(days=7*i)].values[0]
        idv=[x for x in master.columns if x[:4]=='prev']
        idv+=['hpg_reserve_%d'%i,'air_reserve_%d'%i,'num_holidays_%d'%i]
        target='tgt_visitors_%d'%i
        train=master[idv+[target]].dropna(subset=[x for x in master.columns if x[:4]=='prev']+[target]).fillna(0)
        train=train[train.index.get_level_values(1)>=datetime(2017,1,1)]
        model=Lasso(alpha=0.01)
        model.fit(train[idv],train[target])
        print model.score(train[idv],train[target]),model.intercept_, model.coef_
        test_data[target]=model.predict(test_data[idv].fillna(0))
    
    return test_data


In [221]:
cal=store['date_info'].reset_index()
cal['week_start']=cal['calendar_date']-pd.to_timedelta((cal['calendar_date'].dt.weekday+1)%7,unit='D')
cal=cal[~cal['day_of_week'].isin(['Saturday','Sunday'])]
cal=cal.groupby(['week_start'])['holiday_flg'].max().to_frame('num_holidays')

week_start,2015-12-27 00:00:00,2016-01-03 00:00:00,2016-01-10 00:00:00,2016-01-17 00:00:00,2016-01-24 00:00:00,2016-01-31 00:00:00,2016-02-07 00:00:00,2016-02-14 00:00:00,2016-02-21 00:00:00,2016-02-28 00:00:00,...,2017-03-26 00:00:00,2017-04-02 00:00:00,2017-04-09 00:00:00,2017-04-16 00:00:00,2017-04-23 00:00:00,2017-04-30 00:00:00,2017-05-07 00:00:00,2017-05-14 00:00:00,2017-05-21 00:00:00,2017-05-28 00:00:00
num_holidays,1,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [229]:
group_level=['cluster']
weekly_visits=a[a['week_start']>=date(2016,6,1)]
weekly_visits=weekly_visits.groupby(group_level+['week_start'])['visitors'].sum().to_frame('weekly_visits').unstack()

def get_ts_values(ds,period,label,reverse=False):
    weekly_tgts=[]
    for i,j in enumerate(ds.columns.get_level_values(-1)):
        if reverse==True:
            b=ds.iloc[:,i:min(i+period,ds.shape[1])]
        else: 
            b=ds.iloc[:,max(i-period,0):i]
        b.columns=['%s_%d'%(label,x) for x in range(b.shape[1])]
        b.loc[:,'week_start']=j
        weekly_tgts.append(b.set_index('week_start',append=True))
    return pd.concat(weekly_tgts)

tgts=get_ts_values(weekly_visits,6,'tgt_visitors',reverse=True)
pvar=get_ts_values(weekly_visits,6,'prev_visits')
cal_vars=get_ts_values(cal.T,6,'num_holidays',reverse=True).fillna(0)
cal_vars.index=cal_vars.index.get_level_values(1)


In [140]:
c=store['reserve_master'].copy()
c['reserve_date']=c['reserve_date'].astype(np.datetime64)
c['visit_date']=c['visit_date'].astype(np.datetime64)
c['reserve_week']=c['reserve_date']-pd.to_timedelta((c['reserve_date'].dt.weekday+1)%7,unit='D')
c['visit_week']=c['visit_date']-pd.to_timedelta((c['visit_date'].dt.weekday+1)%7,unit='D')
c['interval']=(c['visit_week']-c['reserve_week']).dt.days/7
c=c[(c['interval']>0)&(c['interval']<=6)]
c=c.groupby(group_level+['reserve_week','visit_week','interval','type'])['reserve_visitors'].sum().unstack().reset_index()
c.head()

type,cluster,reserve_week,visit_week,interval,air,hpg
0,0.0,2015-12-27,2016-01-03,1.0,1.098612,157.612393
1,0.0,2015-12-27,2016-01-10,2.0,NaN,50.110882
2,0.0,2015-12-27,2016-01-17,3.0,NaN,14.020572
3,0.0,2015-12-27,2016-01-24,4.0,NaN,7.264030
4,0.0,2016-01-03,2016-01-10,1.0,13.494975,919.652862


In [177]:
def get_reserve_vars(pdate):
    c1=c[(c['reserve_week']<pdate)&(c['visit_week']>=pdate)]
    c2=c1.groupby(group_level+['visit_week'])['hpg'].sum().unstack()
    c2.columns=['hpg_reserve_%d'%((x-pdate).days/7) for x in c2.columns]
    c2['week_start']=pdate
    reserve1=c2.set_index('week_start',append=True)
    c2=c1.groupby(group_level+['visit_week'])['air'].sum().unstack()
    c2.columns=['air_reserve_%d'%((x-pdate).days/7) for x in c2.columns]
    c2['week_start']=pdate
    reserve2=c2.set_index('week_start',append=True)
    return reserve1,reserve2

data=[get_reserve_vars(pdate) for pdate in pd.concat([tgts,pvar]).index.get_level_values(1).drop_duplicates()]
reserve1,reserve2=zip(*data)
reserve1=pd.concat(reserve1)
reserve2=pd.concat(reserve2)


In [259]:
master=pd.concat([tgts,pvar,reserve1,reserve2,cal_vars.reindex(tgts.index, level=1)],axis=1)
test_vars1=weekly_visits.iloc[:,-6:]
test_vars1.columns=['prev_visits_%d'%(5-i) for i in range(6)]
test_reserve_1,test_reserve_2=get_reserve_vars(datetime(2017,4,23))
test_reserve_1.index=test_vars1.index
test_reserve_2.index=test_vars1.index
test_data=pd.concat([test_vars1,test_reserve_1,test_reserve_2],axis=1).fillna(0)

for i in range(6):
    test_data['num_holidays_%d'%i]=cal.loc[date(2017,4,23)+timedelta(days=7*i)].values[0]


In [258]:
for i in range(6):
    print cal.loc[date(2017,4,23)+timedelta(days=7*i)].values[0]


0
1
0
0
0
0


In [244]:
cal.loc[date(2017,4,23)+timedelta(days=7*i)]

num_holidays    0
Name: 2017-04-23 00:00:00, dtype: int64

In [260]:
from sklearn.linear_model import Ridge,Lasso

for i in range(6):
    idv=[x for x in master.columns if x[:4]=='prev']
    #idv+=['hpg_reserve_%d'%i,'air_reserve_%d'%i,'num_holidays_%d'%i]
    idv+=['hpg_reserve_%d'%i,'air_reserve_%d'%i,'num_holidays_%d'%i]
    target='tgt_visitors_%d'%i
    train=master[idv+[target]].dropna(subset=[x for x in master.columns if x[:4]=='prev']+[target]).fillna(0)
    train=train[train.index.get_level_values(1)>=datetime(2017,1,1)]
    model=Lasso(alpha=0.01)
    model.fit(train[idv],train[target])
    print model.score(train[idv],train[target]),model.intercept_, model.coef_
    test_data[target]=model.predict(test_data[idv].fillna(0))
    

0.988085763237 -199.272309199 [  1.63689515e-01   1.12558885e-01   6.07719500e-02   1.93705122e-01
  -5.57346056e-02   3.82563941e-01   4.83187789e-02   2.21742585e+00
  -1.46197319e+02]
0.997296977368 -65.0160173181 [  2.02161603e-01   8.89523886e-02   2.06352610e-01   1.34195687e-01
   1.23545659e-01   2.61379811e-01  -2.78971353e-02   1.72991722e+00
  -8.60965919e+01]
0.997244144966 -58.1160612421 [  0.18930165   0.11636936   0.14146919   0.23108967   0.11704778
   0.24452504  -0.11707317   2.9476794  -37.60477211]
0.997190710134 -39.6544264199 [  0.16933384   0.14484049   0.12543063   0.17077799   0.21572208
   0.25166722  -0.63170811   4.55412663  20.38081374]
0.996719969397 -25.0280086796 [  0.17559306   0.12460976   0.16450066   0.1699254    0.11773443
   0.31074956  -0.02492657   3.17782428 -17.04437644]
0.997317054997 -53.8509977432 [  4.12811348e-01   4.89148077e-02   1.55985036e-01   1.17401728e-01
   1.97440751e-01   7.58862688e-02   7.52341508e+00   6.49565528e+00
   7.763

In [261]:
test_data

,prev_visits_5,prev_visits_4,prev_visits_3,prev_visits_2,prev_visits_1,prev_visits_0,hpg_reserve_0,hpg_reserve_1,hpg_reserve_2,hpg_reserve_3,...,num_holidays_2,num_holidays_3,num_holidays_4,num_holidays_5,tgt_visitors_0,tgt_visitors_1,tgt_visitors_2,tgt_visitors_3,tgt_visitors_4,tgt_visitors_5
cluster,,,,,,,,,,,,,,,,,,,,,
0,1560.471313,1567.344887,1568.251091,1503.895095,1503.415718,1514.511646,2712.600536,888.962835,356.287012,178.525832,...,0,0,0,0,1506.201209,1445.841360,1573.771443,1585.256411,1613.937711,1617.984356
1,7560.592268,7396.530140,7628.909970,7492.454934,7445.872145,7565.798180,8655.573570,1865.644012,1002.153559,497.858035,...,0,0,0,0,7916.012769,7666.985213,7907.646487,7947.803252,8015.766461,7849.289283
2,487.692052,474.098390,480.191834,445.500608,437.469836,482.445021,1146.036472,311.722262,112.458531,56.574649,...,0,0,0,0,529.087462,370.750807,467.713913,490.695253,480.885244,448.575254
3,2166.325859,2221.848560,2214.199746,2047.093669,2055.395791,2068.014626,1770.413294,597.872587,140.850020,74.293846,...,0,0,0,0,2205.227338,2096.529600,2200.827732,2293.288994,2249.297822,2132.157825
4,236.027245,248.467289,217.605848,217.390426,233.414467,241.139854,651.119364,269.078077,98.036887,38.733098,...,0,0,0,0,194.912030,141.517376,212.082668,192.951691,232.561360,183.537446
5,568.014998,558.365981,563.496711,513.353361,528.114936,528.210319,1001.064017,316.290418,113.584914,86.821046,...,0,0,0,0,649.127307,513.991628,571.570676,599.437584,593.194754,541.084838
6,323.799081,350.081347,338.843308,318.701097,331.677440,337.569784,850.091117,323.241840,121.777296,74.251277,...,0,0,0,0,398.273545,260.439960,320.235636,330.506536,360.169528,300.555700
7,303.188484,317.041980,311.939111,299.262976,294.916111,296.988416,551.203317,158.730125,77.622300,30.265167,...,0,0,0,0,246.241816,192.126229,303.014679,291.578077,305.234219,264.191011
8,287.984738,321.597997,287.157223,255.136782,254.298591,253.273724,183.363588,65.901661,15.954564,17.815237,...,0,0,0,0,242.423935,153.028038,248.346793,251.706295,280.141026,222.003148
